In [ ]:
import pandas as pd
import re
import numpy as np
import matplotlib.pyplot as plt

import flask
import csv
from flask import Flask, render_template, request
import difflib
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import random

In [ ]:
import pickle

In [ ]:
#pickle.dump(list1,open('model.pkl','wb'))

In [ ]:
ds = pd.read_excel("/content/Output.xls")

In [ ]:
ds.shape

(886, 11)

In [ ]:
ds = ds.dropna()

In [ ]:
ds.shape

(718, 11)

In [ ]:
def get_important_features(data):
  #important_features = []
  #for i in range(0,data.shape[0]):
    return (str(data['Songs'])+' '+str(data['singer'])+' '+str(data['music'])+' '+str(data['lyricist']))

In [ ]:
ds['combined'] = ds.apply(get_important_features,axis=1)

In [ ]:
ds.head()

,Video ID,Songs,Artists,singer,music,lyricist,Channel,View_Count,Like_Count,Dislike_Count,Comment_Count,combined
0,kckDWrICC4s,NAZAR LAG JAYEGI,"Kamal Raja,Millind Gaba",MilindGaba & KamalRaja,MusicMG,"Ikrar,MillindGaba",T-Series,177575589,1612878.0,106200.0,50701.0,NAZAR LAG JAYEGI MilindGaba & KamalRaja MusicM...
2,HV3-5ivdY88,Bholeynath,"Millind Gaba, Ikka, Pallavi Gaba","MillindGaba,Ikka,PallaviGaba",MillindGaba,Ikka,T-Series,65083886,479157.0,28577.0,11609.0,"Bholeynath MillindGaba,Ikka,PallaviGaba Milli..."
3,zyVaheF55SM,Dil Dooba,"Aishwarya Rai, Akshaye Kumar","SonuNigam,ShreyaGhoshal",RamSampat,Sameer,T-Series,46336027,260590.0,9634.0,6970.0,"Dil Dooba SonuNigam,ShreyaGhoshal RamSampat Sa..."
5,vb7rC7bSDRY,Laal Dupatta,Mika Singh and Anupama Raag,MikaSingh&AnupamaRaag,AnupamaRaag,AnupamaRaag,T-Series,17507816,47293.0,9356.0,1767.0,Laal Dupatta MikaSingh&AnupamaRaag AnupamaRaag...
6,0zF-yRUdEA0,Deedar De,Abhishek Bacchan,"SunidhiChauhan,Krishna","VishalDadlani,ShekharRavjiani",PanchhiJalonvi,T-Series,15314073,64747.0,3308.0,3096.0,"Deedar De SunidhiChauhan,Krishna VishalDadlani..."


In [ ]:
ds['Songs'] = ds['Songs'].str.lower()

In [ ]:
#Convert the text to a matrix of token counts
from sklearn.feature_extraction.text import CountVectorizer
cm = CountVectorizer().fit_transform(ds['combined'])

In [ ]:
#Get the cosine similarity matrix from the count matrix
from sklearn.metrics.pairwise import cosine_similarity
cs = cosine_similarity(cm)
#Print the cosine similarity
print(cs)

[[1.         0.2236068  0.         ... 0.         0.         0.        ]
 [0.2236068  1.         0.         ... 0.         0.         0.        ]
 [0.         0.         1.         ... 0.12909944 0.12309149 0.        ]
 ...
 [0.         0.         0.12909944 ... 1.         0.         0.3354102 ]
 [0.         0.         0.12309149 ... 0.         1.         0.        ]
 [0.         0.         0.         ... 0.3354102  0.         1.        ]]


In [ ]:
pickle.dump(ds,open('modelds.pkl','wb'))
pickle.dump(cs,open('modelcs.pkl','wb'))

In [ ]:
modelds = pickle.load(open('modelds.pkl','rb'))
modelcs = pickle.load(open('modelcs.pkl','rb'))

In [ ]:
!pip install flask-ngrok

In [ ]:
from flask import Flask, request, jsonify, render_template
import pickle
from flask_ngrok import run_with_ngrok
app = Flask(__name__)
run_with_ngrok(app) 
#app = Flask(__name__)

@app.route('/')
def home():
    return render_template('index.html')

@app.route('/submit',methods=['GET','POST'])
def submit():
    ds = pickle.load(open('modelds.pkl', 'rb'))
    cosine_sim = pickle.load(open('modelcs.pkl','rb'))
    song = request.form['song']
    print("Hi")
    ds1 = ds.reset_index()
    print("Hi5")
    indices = pd.Series(ds1.index, index=ds1['Songs'])
    print("Hi1")
    song=song.lower()
    for  title in ds1["Songs"] :
      if(str(song) in str(title)) :
        break
    idx = indices[title]
    print("Hi2")
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]
    l1=[]
    for i in sim_scores:
      l1.append('https://www.youtube.com/embed/'+ds['Video ID'].iloc[i[0]])
    return render_template('index.html', video1=str(l1[0]),video2=str(l1[1]),video3=str(l1[2]))


if __name__ == "__main__":
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://6d9dee6ebe10.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [20/Jun/2021 12:59:41] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [20/Jun/2021 12:59:41] "GET /static/css/style.css HTTP/1.1" 404 -
127.0.0.1 - - [20/Jun/2021 12:59:41] "GET /width=%22250%22 HTTP/1.1" 404 -
127.0.0.1 - - [20/Jun/2021 12:59:41] "GET /width=%22250%22 HTTP/1.1" 404 -
127.0.0.1 - - [20/Jun/2021 12:59:42] "GET /width=%22250%22 HTTP/1.1" 404 -
127.0.0.1 - - [20/Jun/2021 12:59:42] "GET /favicon.ico HTTP/1.1" 404 -


In [ ]:
from google.colab import drive
drive.mount('/content/drive')